# Test LL SSH - Pt2

In [2]:
import pandas as pd
import os
import numpy as np
import statsmodels.discrete.discrete_model as smd
import statsmodels.formula.api as smf

**Import data and format**

In [3]:
# import data
df_raw = pd.read_csv("CyclingPerceptionData-Assignment 1-2024.csv")
df_raw.head()

,ID,sex,age,OwnBikes,NumberOfBikes,HH_Adults_15+,HH_Children_Dummy,NoOf15yrsOlderBikers,BikingForAllReason,BikingForWork,...,Time from MajorBikePath_min,Time from Neasrest BikeLane_Min,Neighbourhood ID,Neighbourhood-Area_SQKM,Neighbourhood-bikeLength_KM,Neighbourhood_Population,Rank_Overall-Quality-Of-Biking,Rank_On-Street-BikeLanes-Quality,Rank_OffStreet-BikeLanes-Quality,Rank_Bike-Parking-Quality
0,40612,0,43,1,2,2,1,1,1,0,...,15,15,124,3.227141,0.763362,22387,3,3,4,5
1,40472,1,29,1,2,4,1,2,1,0,...,5,10,8,2.917358,3.234139,9437,3,1,3,3
2,40897,1,33,1,4,4,1,4,3,0,...,0,0,77,5.960673,1.037788,14830,3,4,4,5
3,40594,1,34,1,3,2,2,2,2,2,...,2,0,43,1.436087,4.510762,13777,4,3,3,4
4,41451,1,24,1,3,5,1,3,1,1,...,25,10,131,37.609169,33.950589,56281,3,4,3,4


In [4]:
# list all provided vars
print('\n'.join(x for x in df_raw.columns if not x.startswith("Rank_")))

ID
sex
age
OwnBikes
NumberOfBikes
HH_Adults_15+
HH_Children_Dummy
NoOf15yrsOlderBikers
BikingForAllReason
BikingForWork
BikingForSchool
BikingForErrndFriends
Num of Recreational-Biker
AccessToMotorVehicle_1Yes-2No-3Unknown
BikeParkingAt Work_1Yes2No3Unknown
How longLivedInNH
Home_1Rent2Own
d3: Home Type
d5: LevelOfEducation
d6: Job Stastus
d8: Income
Time from MajorBikePath_min
Time from Neasrest BikeLane_Min
Neighbourhood ID
Neighbourhood-Area_SQKM
Neighbourhood-bikeLength_KM
Neighbourhood_Population


## Part 1. Binary Probit Model
A dataset: “CyclingPerceptionData-Assignment 1-2024,” is uploaded. 

Use column AB of sheet “Final”, which is of “Rank_Overall-Quality-Of-Biking”. 

Please consider this variable as categorical data. 

Please model the ranking over 2 as a binary Probit model. 

Please select the explanatory variables from this data sheet as relevant you think

For estimation, you can use any software. The report should include at least the following sections:
* Present the model formulations and justification of independent variables. 
* Selection of the best model. 
* Justification of using the explanatory variables 

The report should be professional, not more than 2 pages (singles-space with a 1.25-inch margin on all four sides).

**Prepare data**

In [5]:
# define Y-variable: "Rank_Overall-Quality-Of-Biking" (as categorical)
y_probit = df_raw['Rank_Overall-Quality-Of-Biking'].astype('category')
y_probit

0      3
1      3
2      3
3      4
4      3
      ..
238    5
239    5
240    2
241    3
242    3
Name: Rank_Overall-Quality-Of-Biking, Length: 243, dtype: category
Categories (6, int64): [1, 2, 3, 4, 5, 6]

In [6]:
# convert to binary: y > 2
y_probit_gt2 = (y_probit.astype('int') > 2).astype(int)
y_probit_gt2

0      1
1      1
2      1
3      1
4      1
      ..
238    1
239    1
240    0
241    1
242    1
Name: Rank_Overall-Quality-Of-Biking, Length: 243, dtype: int64

**Select relevant explanatory variables**

See above for list of all non-rank [non-response] features provided in the dataset.

In [1]:
probit_vars = ['age', 'sex', 'BikingForWork', 'Num of Recreational-Biker', 
               'AccessToMotorVehicle_1Yes-2No-3Unknown', 
               'Neighbourhood-bikeLength_KM', 'Time from MajorBikePath_min']

In [17]:
probit_mod = smd.Probit(y_probit_gt2, 
                        df_raw[probit_vars]
                       )
probit_res = probit_mod.fit()
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.501260
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      236
Method:                                       MLE   Df Model:                            6
Date:                            Fri, 23 Feb 2024   Pseudo R-squ.:                 0.01382
Time:                                    21:51:06   Log-Likelihood:                -121.81
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                    0.7553
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
age                                        0.0034      0.004      0.768      0.443      -0.005       0.012
sex                                        0.1014      0.188      0.539      0.590      -0.267       0.470
BikingForWork                              0.3989      0.150      2.665      0.008       0.106       0.692
Num of Recreational-Biker                  0.0015      0.095      0.016      0.987      -0.185       0.188
AccessToMotorVehicle_1Yes-2No-3Unknown  2.961e-05      0.069      0.000      1.000      -0.135       0.136
Neighbourhood-bikeLength_KM                0.0282      0.017      1.688      0.091      -0.005       0.061
Time from MajorBikePath_min                0.0176      0.013      1.398      0.162      -0.007       0.042
==========================================================================================================
"""

In [ ]:
# binary_probit = smf.probit(formula, data = df).fit()
# binary_probit.summary()

## Part 2. Binary Logit Model:
A dataset: “CyclingPerceptionData-Assignment 1-2024,” is uploaded. 

Use column AB of sheet “Final”, which is of “Rank_On-Street-BikeLanes-Quality”. 

Please consider this variable as categorical data. 

Please model the ranking over 1 as a binary Logit model. 

Please select the explanatory variables from this data sheet as relevant you think

For estimation, you can use any software. The report should include at least the following sections:
* Present the model formulations and justification of independent variables. 
* Selection of the best model. 
* Justification of using the explanatory variable

The report should be professional, not more than 2 pages (singles-space with a 1.25-inch margin on all four sides).

**Prepare data**

In [6]:
# define Y-variable: "Rank_On-Street-BikeLanes-Quality" (as categorical)
y_logit = df_raw['Rank_On-Street-BikeLanes-Quality'].astype('category')
y_logit

0      3
1      1
2      4
3      3
4      4
      ..
238    5
239    5
240    2
241    4
242    4
Name: Rank_On-Street-BikeLanes-Quality, Length: 243, dtype: category
Categories (6, int64): [1, 2, 3, 4, 5, 6]

In [7]:
# convert to binary: Y > 1
y_logit_gt1 = (y_logit.astype(int) > 1).astype(int)
y_logit_gt1

0      1
1      0
2      1
3      1
4      1
      ..
238    1
239    1
240    1
241    1
242    1
Name: Rank_On-Street-BikeLanes-Quality, Length: 243, dtype: int64

**Select relevant explanatory variables**

See above for list of all non-rank [non-response] features provided in the dataset.

In [ ]:
# binary_logistic = smf.logit(formula, data = df).fit()
# binary_logistic.summary()